In [7]:
from typing import Optional

from returns.maybe import maybe
from pydantic import BaseModel

class Address(BaseModel):
    street_one: str
    street_two: Optional[str]
    city: str
    state: str

class User(BaseModel):
    username: str
    password: str
    address: Optional[Address]

bob = User(username="bob", password="secret")
frank = User(
    username="fisherman",
    password="master-baiter",
    address=Address(
        street_one="123 Lake Dr.",
        city="Somewhere",
        state="Arklamo"
    )
)

## Maybe useful

This is a super trivial example.
The `Maybe` monad isn't super useful because python programmers are quite handy at dealing with `None` values.
However, I have used it because I love how it provides an *explicit* interface to tell my fellow developers that this data container might be a null value.
In the following example, it would be really easy to check for the address, but you can imagine more complex scenarios where this might be useful.
Also keep in mind that this is just the edge of the rabbit hole...

In [19]:
def get_user_address(user: User) -> str:
    return user.address

def format_address(addr: Address) -> str:
    return f"{addr.street_one}\n{addr.street_two or ''}\n{addr.city}\n{addr.state}\n"

for user in [frank, bob]:
    print(f"{user.username}\n{format_address(get_user_address(user))}\n")

fisherman
123 Lake Dr.

Somewhere
Arklamo




AttributeError: 'NoneType' object has no attribute 'street_one'

## Background

As a side note, you can't escape these monad "containers" in Haskell.
Once a value ends up in a container, you have to keep working in that context.
Haskell also enforces these rules *at compile time*, and it is *strongly typed*.
Haskell enforces functional "purity" at the compiler-level; python does no such thing!
A pure function is one that doesn't rely on any external state or IO.
No matter how many times you call a pure function, it will always return the same output with a given input.
Think `sum([4, 2])` versus `get_input("your name')`.
Pure functions don't raise exceptions, either!
The `returns` developers have done a fantastic job of bringing these features to Python, but you can't change the fact that it's still python.
These annotations are more semantic guidelines than rules.
They can be checked with `mypy`, but at the end of the day, you will still be debugging runtime issues.
It's just the nature of the beast.

### Maybe container

`maybe` wraps a value in a container.
This container exposes a few methods which we can use to compose functions together.
This style of programming is most useful if you think of a series of function calls as a way of slowly building an end result.

```python
def post(my_api_endpoint, payload):
    data = call_db()
    validated = validate_payload(payload)
    return process_rows(data, validated)
```

versus
```python
def post(my_api_endpoint, payload):
    return result.from_iterable(
        [call_db(), validate_payload(payload)]
    ).map(
        process_rows
    ).unwrap()
```

You really have to wrap your head around the idea of things as a series of calls in a pipeline that feeds data from one call to the next.
The "railway" aspect of this is that these chained calls are designed to short-circuit when they are made using special containers.

- `maybe` wraps the returned address in a `Maybe` container.
- `format_address` is a "pure" function.

The last line says, "Call `get_user_address` with `user`.
If the value is `Some[Address]`, pass the output to `format_address`.
`value_or` returns the wrapped value of a `Some` container (*you can't do this in Haskell!).

If the value is `Nothing`, skip `format_address`.
`value_or` returns the value of the given parameter in the `Nothing` case, for instance `""`.

In [17]:
@maybe
def get_user_address(user: User) -> str:
    return user.address

def format_address(addr: Address) -> str:
    return f"{addr.street_one}\n{addr.street_two or ''}\n{addr.city}\n{addr.state}\n"

for user in [frank, bob]:
    print(get_user_address(user).map(format_address).value_or(""))

123 Lake Dr.

Somewhere
Arklamo


